In [27]:
from glob import glob
from os.path import join, split, splitext, dirname
import pandas as pd
from scipy.stats import norm
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from scipy import stats

In [17]:
base_path = '/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/'
input_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')
output_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')

M_vals = [ 8, 16, 32, 64 ]

In [59]:
raw_model_df.keys()

Index(['means', 'seed', 'freq_sds', 'newton_step_norm', 'scipy_opt_result',
       'reference_means', 'reference_freq_sds', 'M',
       'reference_newton_step_norm', 'reference_scipy_opt_result'],
      dtype='object')

In [78]:
def RepList(x, n):
    return [x for _ in range(n)]


def GetCoverageDataframe(filename, M):
    raw_model_df = pd.read_pickle(filename)

    # The final directory is the model name
    model_name = split(os.path.dirname(filename))[-1]

    num_runs = raw_model_df.shape[0]

    means = raw_model_df['means'].to_numpy()
    seeds = raw_model_df['seed'].to_numpy()
    freq_sds = raw_model_df['freq_sds'].to_numpy()
    assert(len(means) == len(freq_sds))
    num_runs = len(means) 
    param_dim = len(means[0]) 

    # A for loop is not the most efficient or elegant but it will let me make sure
    # everything lines up correctly
    model_dict = {
        'seed': [],
        'param': [],
        'mean': [],
        'freq_sd': []
    }
    for i in range(num_runs):
        assert(len(means[i]) == len(freq_sds[i]))
        model_dict['seed'].append(RepList(seeds[i], param_dim))
        model_dict['param'].append(np.arange(param_dim))
        model_dict['mean'].append(means[i])
        model_dict['freq_sd'].append(freq_sds[i])

    model_df = pd.DataFrame()
    for k,v in model_dict.items():
        model_df[k] = np.hstack(v)
    model_df['model'] = model_name
    model_df['num_draws'] = M
    
    return model_df

In [82]:
M = 8
for M in M_vals:
    print(f'Loading for {M} draws')
    coverage_filenames = glob(join(input_path, f'M_{M}', '*', '*.pkl'))
    assert(len(coverage_filenames) > 0)

    model_names = [ split(os.path.dirname(filename))[-1]
                    for filename in coverage_filenames ]

    model_dfs = [ GetCoverageDataframe(filename, M) for filename in coverage_filenames ] 

model_df = pd.concat(model_dfs)

Loading for 8 draws
Loading for 16 draws
Loading for 32 draws
Loading for 64 draws


In [83]:
model_df

,seed,param,mean,freq_sd,model,num_draws
0,1000,0,-0.822654,0.095340,radon_no_pool_chr,64
1,1000,1,-1.569717,0.039489,radon_no_pool_chr,64
2,1000,2,0.072111,0.099980,radon_no_pool_chr,64
3,1000,3,0.060569,0.083363,radon_no_pool_chr,64
4,1000,4,0.027611,0.093779,radon_no_pool_chr,64
...,...,...,...,...,...,...
16995,1099,165,0.155888,0.126451,hiv,64
16996,1099,166,-0.113512,0.128015,hiv,64
16997,1099,167,0.003534,0.126096,hiv,64
16998,1099,168,-2.614673,0.205410,hiv,64
